In [60]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Configure CRUD methods for AAC database
from aac_crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Authentication credentials
username = "aacuser"
password = "passWord"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Grazioso Salvare AAC Dashboard by Janai Cano')

# Adding in Grazioso Salvare’s logo
image_filename = 'gs_logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # html.Div(id='hidden-div', style={'display':'none'}),
    # Header
    # Generates clickable logo image to url 'snhu.edu'
    html.Center(dcc.Link(
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                 style={'height': '90px', 'width': '90px'}),
            href="www.snhu.edu"
    )),
    html.Center(html.B(html.H2('CS-340 Dashboard by Janai Cano'))),
    html.Hr(),
    html.Div([
        # Dropdown menu
        dcc.Dropdown(
            id='dropdown-filter',
            options=[
                {'label': 'Water Rescue', 'value': 'WATER'},
                {'label': 'Disaster Rescue', 'value': 'DISASTER'},
                {'label': 'Mountain Rescue', 'value': 'MOUNTAIN'},
                {'label': 'All Animals', 'value': 'ALL'},
            ],
            value='ALL'
        ),
    ]),
    html.Hr(),
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        data=df.to_dict('records'),
        row_selectable='single',  # allows one row to be selected at a time
        sort_action='native',  # allows for sorting by column
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        # styles the text and background of the table for readability
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto',
            'color': 'black',
            'backgroundColor': 'white'
        },
        # styles every other row an off-white color for readability
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(220, 220, 220)',
            }
        ],
        # styles the header row for readability
        style_header={
            'backgroundColor': 'rgb(210, 210, 210)',
            'color': 'black',
            'fontWeight': 'bold'
        },
        # enables pages and horizontal and vertical scrolling
        page_size=50,
        style_table={
            'overflowY': 'auto',
            'overflowX': 'auto'
        },
        # makes sure the headers stay at the top as user scrolls down
        fixed_rows={'headers': True},
        # styles each cell to align left, plus width of cells for readability
        style_cell={
            'textAlign': 'left',
            'minWidth': 95, 'maxWidth': 150, 'width': 120
        }
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',

                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])


#############################################
# Interaction Between Components / Controller
#############################################

# Updates the data table and pie chart based on dropdown menu selection
@app.callback(Output('datatable-id', 'data'), [Input('dropdown-filter', 'value')])
def update_dashboard(filter_type):
    # Default selection is 'All', no filter is applied
    if filter_type == 'ALL':
        dff = df
    # See Grazioso Salvare breed restrictions & requirements for each filter
    elif filter_type == 'MOUNTAIN':
        dff = pd.DataFrame(db.read(
            {
                "breed": {
                    "$in": [
                        "German Shepherd",
                        "Alaskan Malamute",
                        "Old English Sheepdog",
                        "Siberian Husky",
                        "Rottweiler"
                    ]
                },
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {
                    "$gte": 26.0
                },
                "$and": [
                    {
                        "age_upon_outcome_in_weeks": {
                            "$lte": 156.0
                        }
                    }
                ]
            }
        ))
        dff.drop(columns=['_id'], inplace=True)
    elif filter_type == 'WATER':
        dff = pd.DataFrame(db.read(
            {
                "breed": {
                    "$in": [
                        "Chesapeake Bay Retriever",
                        "Labrador Retriever Mix",
                        "Newfoundland"
                    ]
                },
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {
                    "$gte": 26.0
                },
                "$and": [
                    {
                        "age_upon_outcome_in_weeks": {
                            "$lte": 156.0
                        }
                    }
                ]
            }
        ))
        dff.drop(columns=['_id'], inplace=True)
    elif filter_type == 'DISASTER':
        dff = pd.DataFrame(db.read(
            {
                "breed": {
                    "$in": [
                        "Doberman Pinscher",
                        "German Shepherd",
                        "Golden Retriever",
                        "Bloodhound",
                        "Rottweiler"
                    ]
                },
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {
                    "$gte": 20.0
                },
                "$and": [
                    {
                        "age_upon_outcome_in_weeks": {
                            "$lte": 300.0
                        }
                    }
                ]
            }
        ))
        dff.drop(columns=['_id'], inplace=True)
    # Formatting the results for the data table
    data = dff.to_dict('records')
    return data


# Display a pie chart of animal per breed for current filter
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "data")])
def update_graphs(view_data):
    # Defaults to show all breeds
    if view_data is None:
        dff = df
    else:
        dff = pd.DataFrame.from_dict(view_data)
    # Formatting the pie chart
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    figure = px.pie(dff, names=names, values=values)
    figure.update_traces(textposition='inside')
    figure.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')

    return dcc.Graph(figure=figure)


# This callback will update the geo-location chart for the selected data entry
# For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "data"),
     Input('datatable-id', 'selected_rows')])
def update_map(view_data, selected_rows):
    dff = pd.DataFrame.from_dict(view_data)
    # Because we only allow single row selection, the list can
    # be converted to a row index here
    # if no row is selected, no map is shown
    if view_data is None or selected_rows is None:
        row = 0
    else:
        row = selected_rows[0]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                # Column 13 and 14 define the grid-coordinates
                # Column 4 defines the breed for the animal
                # Column 9 defines the name of the animal
                # Column 3 defines the type of the animal
                dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                          children=[
                              dl.Tooltip(dff.iloc[row, 4]),
                              dl.Popup([
                                  html.H1(dff.iloc[row, 9]),
                                  html.P(dff.iloc[row, 3])
                              ])
                          ])
            ])
    ]


app.run_server(debug=True)


Connected to Austin Animal Center Database
Dash app running on http://127.0.0.1:18973/
